In [1]:
import numpy as np

def LJ(r): #define Lennard-Jones Potential 
    epsilon=1 #depth of potential well
    sigma=1 #distance at which the potential crosses zero
    r6=r**6 
    r12=r6*r6
    return 4*epsilon*(sigma/r12- sigma/r6)

In [2]:
def total_LJ(pos): #calculate total energy of N number of atoms 
    E=0
    N_atom= int(len(pos)/3) 
    
    for i in range(N_atom-1): 
        for j in range(i+1, N_atom):
            Pos1= pos[i*3:(i+1)*3]
            Pos2= pos[j*3:(j+1)*3]
            distance=np.linalg.norm(Pos1-Pos2)
            
            E += LJ(distance)
    return E

In [10]:
def init_pos(N,L=5): #generate random positions for N number of atoms 
    return L*np.random.random_sample((N*3))

In [90]:
def jacobian(pos):
    N_atom = int(len(pos)/3) 
    ljforce = np.zeros([N_atom, 3]) #generate array to store forces
                 
    for i in range(N_atom): 
        pos0= pos[i*3:(i+1)*3] #position of central atom
        for j in range(N_atom):
            pos1= pos[j*3:(j+1)*3]
            if not any(pos0==pos1): #check that   
                distance=np.linalg.norm(pos0-pos1)
                r14 =distance**14
                r8 = distance**8
                x= 48/r14
                y=24/r8
                rnext = np.dot(y-x, (pos0-pos1))
                ljforce[i] += rnext #add force to array 

    return ljforce.flatten() #return array collapsed into one dimension 

In [91]:
from scipy.optimize import minimize
import time

start_time=time.time()
fx_list=[] #store function values 
x_list=[] #store x values 

def ground_state(N_atom, method, N_attempts): #calculate ground state potential of N number of atoms 
    start_time= time.time()
    for i in range(N_attempts):
        pos= init_pos(N_atom)
        results=minimize(total_LJ, pos, method=method, jac=jacobian, tol=1e-4) #use scipy minimize function to find lowest function value 
        fx_list.append(results.fun)
        x_list.append(results.x)
        if i%10==0: #for every 10 steps, record value at that point and time 
            end_time=time.time()
            total_time= end_time-start_time #calculate execution time 
            print("Method: ", method, 'Step #: ', i, '  Value:', results.fun, "Time: ", total_time, "sec")    
        
    print ("Ground state potential is:", min(fx_list))

In [92]:
ground_state(3, 'BFGS', 50)

Method:  BFGS Step #:  0   Value: -2.9999999999973532 Time:  0.01602792739868164 sec
Method:  BFGS Step #:  10   Value: -2.999999999999993 Time:  0.1162109375 sec
Method:  BFGS Step #:  20   Value: -2.9999999999998375 Time:  0.20913481712341309 sec
Method:  BFGS Step #:  30   Value: -2.9999999999980718 Time:  0.3126859664916992 sec
Method:  BFGS Step #:  40   Value: -2.9999999999999325 Time:  0.4142029285430908 sec
Ground state potential is: -3.0


In [93]:
ground_state(20,'BFGS', 100)

Method:  BFGS Step #:  0   Value: -67.8296724968687 Time:  2.118759870529175 sec
Method:  BFGS Step #:  10   Value: -53.02430840674744 Time:  24.984334707260132 sec
Method:  BFGS Step #:  20   Value: -67.26115129790182 Time:  49.505085945129395 sec
Method:  BFGS Step #:  30   Value: -71.06253552541175 Time:  73.25586199760437 sec
Method:  BFGS Step #:  40   Value: -66.23955825694425 Time:  101.57443404197693 sec
Method:  BFGS Step #:  50   Value: -73.88241601743742 Time:  121.0724847316742 sec
Method:  BFGS Step #:  60   Value: -73.56326540777489 Time:  150.4079179763794 sec
Method:  BFGS Step #:  70   Value: -67.06148455066172 Time:  176.8175778388977 sec
Method:  BFGS Step #:  80   Value: -68.88721415580227 Time:  203.01572608947754 sec
Method:  BFGS Step #:  90   Value: -71.3587615088627 Time:  226.9172489643097 sec
Ground state potential is: -75.28121415137447
